In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 설치 (최초 1회만 필요)
# =========================================================
!pip -q install optuna tqdm ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.7 MB/s eta 0:00:00


# 3-fold

In [ ]:
# Import
# =========================================================
import os, json, warnings, math, random, pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    average_precision_score, roc_auc_score, f1_score, accuracy_score,
    precision_recall_curve
)
import optuna

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

# =========================================================
# 파일 경로
# =========================================================
FILE_PATHS = {
    "Amazon":  '/content/drive/MyDrive/1014/data/new_amazon.csv',
    "Coursera":'/content/drive/MyDrive/1014/data/new_coursera.csv',
    "Audible": '/content/drive/MyDrive/1014/data/new_audible.csv',
    "Hotel":   '/content/drive/MyDrive/1014/data/new_hotel.csv'
}

# =========================================================
# S1 피처 (그대로 사용, 없는 컬럼은 자동 제외)
# =========================================================
S1_FEATURES = {
    "Amazon":  ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Price','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Is_Photo','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Coursera":['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Num_of_Reviews','Num_of_Enrolled','Num_of_top_instructor_courses','Num_of_top_instructor_learners','Text_Length','Valence','Arousal','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Audible": ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Hotel":   ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth','Is_Photo','Hotel_Grade','Employee_Friendliness_Score','Facility_Score','Cleanliness_Score','Comfort_Score','Value_For_Money_Score','Location_Score']
}

# =========================================================
# Config
# =========================================================
TARGET_COLUMN     = 'binary_helpfulness'
TEST_SPLIT_RATIO  = 0.2
RANDOM_STATE      = 42
N_TRIALS          = 50

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device = {DEVICE}")

def set_seed(seed=RANDOM_STATE):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(RANDOM_STATE)

# =========================================================
# Utils
# =========================================================
def _make_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    num = df.apply(pd.to_numeric, errors='coerce')
    med = num.median()
    return num.fillna(med)

def find_best_threshold(y_true, prob):
    precision, recall, thresholds = precision_recall_curve(y_true, prob)
    thresholds = np.concatenate([thresholds, [1.0]])
    f1s = (2 * precision * recall) / np.clip(precision + recall, 1e-9, None)
    idx = int(np.nanargmax(f1s))
    return float(thresholds[idx]), float(f1s[idx])

class NumpyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1,1)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

class MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim=256, depth=2, dropout=0.2):
        super().__init__()
        layers=[]; d=in_dim
        for _ in range(depth):
            layers += [nn.Linear(d, hidden_dim), nn.ReLU(), nn.Dropout(dropout)]
            d = hidden_dim
        layers += [nn.Linear(d, 1)]  # logits
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

def train_one_model(X_tr, y_tr, X_va, y_va, params):
    """한 번의 학습: 조기종료(early stopping)로 best PRAUC 모델 반환"""
    batch_size   = params.get("batch_size", 1024)
    lr           = params.get("lr", 1e-3)
    weight_decay = params.get("weight_decay", 1e-5)
    hidden_dim   = params.get("hidden_dim", 256)
    depth        = params.get("depth", 2)
    dropout      = params.get("dropout", 0.2)
    max_epochs   = params.get("max_epochs", 50)
    patience     = params.get("patience", 5)

    model = MLP(X_tr.shape[1], hidden_dim, depth, dropout).to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.BCEWithLogitsLoss()

    dl_tr = DataLoader(NumpyDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True, drop_last=False)
    dl_va = DataLoader(NumpyDataset(X_va, y_va), batch_size=4096, shuffle=False, drop_last=False)

    best_pr_auc = -1.0
    best_state  = None
    bad_epochs  = 0

    for ep in range(1, max_epochs+1):
        model.train()
        for xb, yb in dl_tr:
            xb = xb.to(DEVICE); yb = yb.to(DEVICE)
            logit = model(xb)
            loss  = crit(logit, yb)
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            opt.step()

        # ---- validation (PR_AUC)
        model.eval(); probs=[]; ys=[]
        with torch.no_grad():
            for xb, yb in dl_va:
                xb = xb.to(DEVICE)
                logit = model(xb)
                prob = torch.sigmoid(logit).cpu().numpy().ravel()
                probs.append(prob)
                ys.append(yb.numpy().ravel())
        probs = np.concatenate(probs); ys = np.concatenate(ys)
        pr = average_precision_score(ys, probs)

        # early stopping
        if pr > best_pr_auc + 1e-6:
            best_pr_auc = pr
            best_state  = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad_epochs  = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                break

    # load best
    if best_state is not None:
        model.load_state_dict(best_state)
    return model, best_pr_auc

# =========================================================
# 메인 파이프라인 (플랫폼 단위)
# =========================================================
def run_s1_pipeline(platform, csv_path, features):
    print("\n" + "="*60)
    print(f"▶ Platform: {platform}")
    print("="*60)

    # 1) 데이터 로드 & 전처리
    df = pd.read_csv(csv_path)
    assert TARGET_COLUMN in df.columns, f"{TARGET_COLUMN} 없음"
    labels = df[TARGET_COLUMN].astype(int).values

    exists = [c for c in features if c in df.columns]
    if len(exists)==0:
        raise ValueError("사용 가능한 S1 피처가 없습니다.")
    X_all = _make_numeric_df(df[exists]).to_numpy()

    # Stratified split
    idx = np.arange(len(df))
    tr_idx, te_idx = train_test_split(
        idx, test_size=TEST_SPLIT_RATIO, random_state=RANDOM_STATE, stratify=labels
    )
    X_train_raw, X_test_raw = X_all[tr_idx], X_all[te_idx]
    y_train, y_test = labels[tr_idx], labels[te_idx]

    # 스케일링(MLP 필수) — train 기준으로 fit
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_raw)
    X_test  = scaler.transform(X_test_raw)
    print(f"✅ Train={len(y_train)}, Test={len(y_test)} | d={X_train.shape[1]}")

    # 2) Optuna (PR_AUC 최대화) — 3-fold CV
    def objective(trial):
        params = {
            "hidden_dim": trial.suggest_categorical("hidden_dim", [128, 256, 512, 768]),
            "depth": trial.suggest_int("depth", 1, 4),
            "dropout": trial.suggest_float("dropout", 0.0, 0.5),
            "lr": trial.suggest_float("lr", 1e-4, 5e-3, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 1e-7, 1e-3, log=True),
            "batch_size": trial.suggest_categorical("batch_size", [256, 512, 1024, 2048]),
            "max_epochs": 50,
            "patience": 5,
        }

        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
        pr_aucs = []
        # 각 fold마다 scaler 재적용(데이터 누수 방지)
        for tr, va in skf.split(X_train, y_train):
            # fold별로 따로 스케일러를 맞추는게 엄밀하지만, 위에서 이미 전체 train에 맞췄으므로
            # 엄격 모드로 하려면 다음 두 줄 사용:
            # _sc = StandardScaler().fit(X_train[tr]); X_tr = _sc.transform(X_train[tr]); X_va = _sc.transform(X_train[va])
            X_tr, X_va = X_train[tr], X_train[va]
            y_tr, y_va = y_train[tr], y_train[va]

            model, val_pr = train_one_model(X_tr, y_tr, X_va, y_va, params)
            pr_aucs.append(val_pr)

            # 메모리 tidy
            del model
            torch.cuda.empty_cache()

        return float(np.mean(pr_aucs))

    study = optuna.create_study(direction="maximize")
    with tqdm(total=N_TRIALS, desc=f"Optuna Tuning [{platform}]", unit="trial") as pbar:
        def cb(study, trial):
            pbar.update(1)
        study.optimize(objective, n_trials=N_TRIALS, callbacks=[cb])

    best_params = study.best_params
    print("🧪 Best Params (MLP):", best_params)

    # 3) 최종 학습 (train 내부 10%를 val로 사용해 early stopping)
    sub_tr, val = train_test_split(
        np.arange(len(y_train)), test_size=0.1, random_state=RANDOM_STATE, stratify=y_train
    )
    X_sub, y_sub = X_train[sub_tr], y_train[sub_tr]
    X_val, y_val = X_train[val],   y_train[val]

    final_model, _ = train_one_model(X_sub, y_sub, X_val, y_val, {**best_params})

    # 4) 평가 + threshold
    final_model.eval()
    with torch.no_grad():
        tr_prob = torch.sigmoid(final_model(torch.tensor(X_train, dtype=torch.float32).to(DEVICE))).cpu().numpy().ravel()
        te_prob = torch.sigmoid(final_model(torch.tensor(X_test,  dtype=torch.float32).to(DEVICE))).cpu().numpy().ravel()

    best_th, _ = find_best_threshold(y_train, tr_prob)
    te_pred = (te_prob >= best_th).astype(int)

    metrics = {
        "Accuracy": float(accuracy_score(y_test, te_pred)),
        "PR_AUC":   float(average_precision_score(y_test, te_prob)),
        "ROC_AUC":  float(roc_auc_score(y_test, te_prob)),
        "F1_score": float(f1_score(y_test, te_pred)),
        "Best_Threshold": float(best_th)
    }
    print("=== Test Metrics (MLP) ===", metrics)

    # 5) 저장
    save_dir = f"/content/drive/MyDrive/1014/result_mlp/{platform}"
    os.makedirs(save_dir, exist_ok=True)

    # 확률/예측
    pd.DataFrame({
        "index": te_idx,
        "s1_pred_proba": te_prob,
        "y_true": y_test,
        "y_pred_at_best_th": te_pred
    }).to_csv(f"{save_dir}/s1_pred_proba.csv", index=False)

    # 메트릭 + 설정
    with open(f"{save_dir}/results.json", "w") as f:
        json.dump({
            **metrics,
            "features_used": exists,
            "random_state": RANDOM_STATE,
            "best_params": best_params
        }, f, indent=2)

    # 모델 & 스케일러 저장
    torch.save(final_model.state_dict(), f"{save_dir}/mlp_model.pt")
    with open(f"{save_dir}/scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)

    print(f"📁 Results saved in {save_dir}")
    print(f"🧠 Model: {save_dir}/mlp_model.pt  |  Scaler: {save_dir}/scaler.pkl")

# =========================================================
# 전체 실행 (플랫폼 루프도 notebook tqdm)
# =========================================================
for platform, path in tqdm(FILE_PATHS.items(), desc="전체 플랫폼 진행 (MLP)", unit="platform"):
    run_s1_pipeline(platform, path, S1_FEATURES[platform])

# 5-fold

In [3]:
# =========================================================
# MLP + Optuna + 5-Fold CV (scaler 누수 방지)
# =========================================================
import os, json, warnings, math, random, pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    average_precision_score, roc_auc_score, f1_score, accuracy_score,
    precision_recall_curve
)
import optuna

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

# =========================================================
# Config
# =========================================================
FILE_PATHS = {
    #"Amazon":  '/content/drive/MyDrive/data/new_amazon.csv',
    "Coursera":'/content/drive/MyDrive/1014/data/new_coursera.csv',
    "Audible": '/content/drive/MyDrive/1014/data/new_audible.csv',
    "Hotel":   '/content/drive/MyDrive/1014/data/new_hotel.csv'
}

S1_FEATURES = {
    "Amazon":  ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Price','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Is_Photo','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Coursera":['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Num_of_Reviews','Num_of_Enrolled','Num_of_top_instructor_courses','Num_of_top_instructor_learners','Text_Length','Valence','Arousal','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Audible": ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Hotel":   ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth','Is_Photo','Hotel_Grade','Employee_Friendliness_Score','Facility_Score','Cleanliness_Score','Comfort_Score','Value_For_Money_Score','Location_Score']
}

TARGET_COLUMN     = 'binary_helpfulness'
TEST_SPLIT_RATIO  = 0.2
RANDOM_STATE      = 42
N_TRIALS          = 50
N_SPLITS          = 5   # ✅ 5-fold CV

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device = {DEVICE}")

def set_seed(seed=RANDOM_STATE):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(RANDOM_STATE)

# =========================================================
# Utils
# =========================================================
def _make_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    num = df.apply(pd.to_numeric, errors='coerce')
    med = num.median()
    return num.fillna(med)

def find_best_threshold(y_true, prob):
    precision, recall, thresholds = precision_recall_curve(y_true, prob)
    thresholds = np.concatenate([thresholds, [1.0]])
    f1s = (2 * precision * recall) / np.clip(precision + recall, 1e-9, None)
    idx = int(np.nanargmax(f1s))
    return float(thresholds[idx]), float(f1s[idx])

class NumpyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1,1)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

class MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim=256, depth=2, dropout=0.2):
        super().__init__()
        layers=[]; d=in_dim
        for _ in range(depth):
            layers += [nn.Linear(d, hidden_dim), nn.ReLU(), nn.Dropout(dropout)]
            d = hidden_dim
        layers += [nn.Linear(d, 1)]  # logits
        self.net = nn.Sequential(*layers)
    def forward(self, x): return self.net(x)

def train_one_model(X_tr, y_tr, X_va, y_va, params):
    batch_size   = params.get("batch_size", 1024)
    lr           = params.get("lr", 1e-3)
    weight_decay = params.get("weight_decay", 1e-5)
    hidden_dim   = params.get("hidden_dim", 256)
    depth        = params.get("depth", 2)
    dropout      = params.get("dropout", 0.2)
    max_epochs   = params.get("max_epochs", 50)
    patience     = params.get("patience", 5)

    model = MLP(X_tr.shape[1], hidden_dim, depth, dropout).to(DEVICE)
    opt   = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    crit  = nn.BCEWithLogitsLoss()

    dl_tr = DataLoader(NumpyDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True)
    dl_va = DataLoader(NumpyDataset(X_va, y_va), batch_size=4096, shuffle=False)

    best_pr_auc = -1.0
    best_state  = None
    bad_epochs  = 0

    for ep in range(1, max_epochs+1):
        model.train()
        for xb, yb in dl_tr:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logit = model(xb)
            loss  = crit(logit, yb)
            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            opt.step()

        # validation (PR_AUC)
        model.eval(); probs=[]; ys=[]
        with torch.no_grad():
            for xb, yb in dl_va:
                xb = xb.to(DEVICE)
                prob = torch.sigmoid(model(xb)).cpu().numpy().ravel()
                probs.append(prob); ys.append(yb.numpy().ravel())
        probs = np.concatenate(probs); ys = np.concatenate(ys)
        pr = average_precision_score(ys, probs)

        if pr > best_pr_auc + 1e-6:
            best_pr_auc = pr
            best_state  = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad_epochs  = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience: break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model, best_pr_auc

# =========================================================
# Main Pipeline
# =========================================================
def run_s1_pipeline(platform, csv_path, features):
    print("\n" + "="*60)
    print(f"▶ Platform: {platform}")
    print("="*60)

    df = pd.read_csv(csv_path)
    assert TARGET_COLUMN in df.columns
    labels = df[TARGET_COLUMN].astype(int).values

    exists = [c for c in features if c in df.columns]
    if len(exists)==0: raise ValueError("사용 가능한 S1 피처 없음")
    X_all = _make_numeric_df(df[exists]).to_numpy()

    idx = np.arange(len(df))
    tr_idx, te_idx = train_test_split(idx, test_size=TEST_SPLIT_RATIO, random_state=RANDOM_STATE, stratify=labels)
    X_train_raw, X_test_raw = X_all[tr_idx], X_all[te_idx]
    y_train, y_test = labels[tr_idx], labels[te_idx]

    # --- Optuna objective (5-fold CV, fold별 scaler fit) ---
    def objective(trial):
        params = {
            "hidden_dim":   trial.suggest_categorical("hidden_dim", [128, 256, 512, 768]),
            "depth":        trial.suggest_int("depth", 1, 4),
            "dropout":      trial.suggest_float("dropout", 0.0, 0.5),
            "lr":           trial.suggest_float("lr", 1e-4, 5e-3, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 1e-7, 1e-3, log=True),
            "batch_size":   trial.suggest_categorical("batch_size", [256, 512, 1024, 2048]),
            "max_epochs":   50,
            "patience":     5,
        }
        skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
        pr_aucs=[]
        for tr, va in skf.split(X_train_raw, y_train):
            sc = StandardScaler().fit(X_train_raw[tr])
            X_tr, X_va = sc.transform(X_train_raw[tr]), sc.transform(X_train_raw[va])
            y_tr, y_va = y_train[tr], y_train[va]
            model, val_pr = train_one_model(X_tr, y_tr, X_va, y_va, params)
            pr_aucs.append(val_pr)
            del model; torch.cuda.empty_cache()
        return float(np.mean(pr_aucs))

    study = optuna.create_study(direction="maximize")
    with tqdm(total=N_TRIALS, desc=f"Optuna Tuning [{platform}]", unit="trial") as pbar:
        def cb(study, trial): pbar.update(1)
        study.optimize(objective, n_trials=N_TRIALS, callbacks=[cb])

    best_params = study.best_params
    print("🧪 Best Params:", best_params)

    # --- 최종 학습 (train 전체 + scaler) ---
    scaler = StandardScaler().fit(X_train_raw)
    X_train, X_test = scaler.transform(X_train_raw), scaler.transform(X_test_raw)

    sub_tr, val = train_test_split(np.arange(len(y_train)), test_size=0.1, random_state=RANDOM_STATE, stratify=y_train)
    X_sub, y_sub = X_train[sub_tr], y_train[sub_tr]
    X_val, y_val = X_train[val],   y_train[val]

    final_model, _ = train_one_model(X_sub, y_sub, X_val, y_val, {**best_params})

    # --- 평가 ---
    final_model.eval()
    with torch.no_grad():
        tr_prob = torch.sigmoid(final_model(torch.tensor(X_train, dtype=torch.float32).to(DEVICE))).cpu().numpy().ravel()
        te_prob = torch.sigmoid(final_model(torch.tensor(X_test,  dtype=torch.float32).to(DEVICE))).cpu().numpy().ravel()

    best_th, _ = find_best_threshold(y_train, tr_prob)
    te_pred = (te_prob >= best_th).astype(int)

    metrics = {
        "Accuracy": float(accuracy_score(y_test, te_pred)),
        "PR_AUC":   float(average_precision_score(y_test, te_prob)),
        "ROC_AUC":  float(roc_auc_score(y_test, te_prob)),
        "F1_score": float(f1_score(y_test, te_pred)),
        "Best_Threshold": float(best_th)
    }
    print("=== Test Metrics ===", metrics)

    # --- 저장 ---
    save_dir = f"/content/drive/MyDrive/1014/result_mlp5/{platform}"
    os.makedirs(save_dir, exist_ok=True)
    pd.DataFrame({
        "index": te_idx,
        "s1_pred_proba": te_prob,
        "y_true": y_test,
        "y_pred_at_best_th": te_pred
    }).to_csv(f"{save_dir}/s1_pred_proba.csv", index=False)
    with open(f"{save_dir}/results.json", "w") as f:
        json.dump({**metrics,"features_used":exists,"random_state":RANDOM_STATE,"best_params":best_params}, f, indent=2)
    torch.save(final_model.state_dict(), f"{save_dir}/mlp_model.pt")
    with open(f"{save_dir}/scaler.pkl","wb") as f: pickle.dump(scaler,f)

    print(f"📁 Saved in {save_dir}")

# =========================================================
# Run All Platforms
# =========================================================
for platform, path in tqdm(FILE_PATHS.items(), desc="전체 플랫폼 진행 (MLP)", unit="platform"):
    run_s1_pipeline(platform, path, S1_FEATURES[platform])

✅ Device = cuda


전체 플랫폼 진행 (MLP):   0%|          | 0/3 [00:00<?, ?platform/s]


▶ Platform: Coursera


Optuna Tuning [Coursera]:   0%|          | 0/50 [00:00<?, ?trial/s]

🧪 Best Params: {'hidden_dim': 256, 'depth': 3, 'dropout': 0.4100844240198673, 'lr': 0.0009760516391587593, 'weight_decay': 3.726080253609299e-07, 'batch_size': 256}
=== Test Metrics === {'Accuracy': 0.9475656973391547, 'PR_AUC': 0.40514052732582, 'ROC_AUC': 0.8901870524983374, 'F1_score': 0.4299149126735334, 'Best_Threshold': 0.20182031393051147}
📁 Saved in /content/drive/MyDrive/1014/result_mlp5/Coursera

▶ Platform: Audible


Optuna Tuning [Audible]:   0%|          | 0/50 [00:00<?, ?trial/s]

🧪 Best Params: {'hidden_dim': 256, 'depth': 4, 'dropout': 0.19522221566144854, 'lr': 0.0017252356782725706, 'weight_decay': 3.636241307304121e-05, 'batch_size': 512}
=== Test Metrics === {'Accuracy': 0.9045058608452522, 'PR_AUC': 0.3079070677108698, 'ROC_AUC': 0.7797977166445202, 'F1_score': 0.3616103522645579, 'Best_Threshold': 0.192869633436203}
📁 Saved in /content/drive/MyDrive/1014/result_mlp5/Audible

▶ Platform: Hotel


Optuna Tuning [Hotel]:   0%|          | 0/50 [00:00<?, ?trial/s]

🧪 Best Params: {'hidden_dim': 128, 'depth': 3, 'dropout': 0.31045426457642256, 'lr': 0.0014417821799230432, 'weight_decay': 5.660609028019945e-05, 'batch_size': 512}
=== Test Metrics === {'Accuracy': 0.8290598290598291, 'PR_AUC': 0.2168572500779447, 'ROC_AUC': 0.701874458962752, 'F1_score': 0.2717753450737744, 'Best_Threshold': 0.16104565560817719}
📁 Saved in /content/drive/MyDrive/1014/result_mlp5/Hotel
